#  Connectivity ..

In [ ]:
import os

from katana import remote

os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
# ##################################################################

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=6)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

# my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)

In [ ]:
# ##################################################################

In [ ]:

from katana_enterprise.client.rest.v2 import client

# Adjust the number of rows (None: no limit) to limit the result set size.
#
my_client.Limits.set_max_operation_result_rows(10)


#  ---------------------------------------------------------------------------
#  ModuleNotFoundError                       Traceback (most recent call last)
#  Input In [12], in <cell line: 1>()
#  ----> 1 from katana_enterprise.client.rest.v2 import client
#        3 ## Adjust the number of rows (None: no limit) to limit the result set size.
#        4 my_client.Limits.set_max_operation_result_rows(10)
#  
#  ModuleNotFoundError: No module named 'katana_enterprise.client'


#  Load graph ..

In [ ]:

from katana.remote import import_data

import_data.csv(
   my_graph,
      #     
   input_node_path   = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/node_list_2M_full.txt",
   input_edge_path   = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/edge_list_2M_full.txt",
   input_dir         = "gs://katana-demo-datasets/csv-datasets/social/csv/2M",
      #     
   data_delimiter    = "|",
   schema_delimiter  = "|",
      #     
   ids_are_integers  = True,
   )

#  import_data.csv(
#     my_graph,
#        #
#     input_node_path   = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/node_list_blacklist0.01.txt",
#     input_edge_path   = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/edge_list_full.txt",
#     input_dir         = "gs://katana-demo-datasets/csv-datasets/social/csv/100_Million",
#        #
#     data_delimiter    = "|",
#     schema_delimiter  = "|",
#        #
#     ids_are_integers  = True,
#     )

print("--")


In [ ]:
# ##################################################################

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
# my_graph.query("CALL graph.schema() RETURN *")


In [9]:
l_result1 = my_graph.query("""

   MATCH (a) WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """)

display(print(l_result1))

l_result2 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """)

display(print(l_result2))

          0/? [?op/s]

       cnt    label
0  2000000  Account


None

          0/? [?op/s]

         temp   tempCnt
0   AssetConA   9803020
1   AssetConB  20897164
2  FriendConA   4503130
3  FriendConB   5799680


None

In [10]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   LIMIT 1000
   
   """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

# Analytics ..

In [11]:

l_result = my_graph.query("""

   MATCH (acct1) - [ :FriendConB ] -> (acct2) - [ :FriendConB ] -> (acct3)
   WHERE 
      (acct3.userIsBlacklisted="Y" OR acct3.userIsSuspended="Y")
      AND NOT (acct1)-->(acct3)
      
   RETURN acct1.id, count(acct3) 
   LIMIT 100

   """)

# print(l_result)
   #
l_result.table()


          0/? [?op/s]

GridBox(children=(HBox(children=(Text(value='', placeholder='Search...'), Label(value='Count: 100 rows'), HBox…